# A. There are different ways to access the files:

## 1.	Map network drive
You can map this new sharepoint site in a similar way as the old one:
Do this only once:
-	Log in to https://wageningenur4.sharepoint.com/sites/LoobosTeamsite with InternetExplorer (the old one, this step does not work with other browsers)
-	Go to settings (gearbox top right) --> Internet Options --> Security tab --> and add https://wageningenur4.sharepoint.com to the list of trusted sites
-	Go to Windows Explorer --> Home (menu bar) --> Easy Access --> Map as Drive --> Select an unused drive letter (e.g. L:\. At least >= D, as A,B,C are reserved for floppy disks and hard disks), copy in to https://wageningenur4.sharepoint.com/sites/LoobosTeamsite into ‘Folder’ and click Finish
-	The Sharepoint should now be available via the drive letter you selected.

Do this each time you want to use the drive letter:
-	Log in in to https://wageningenur4.sharepoint.com/sites/LoobosTeamsite with InternetExplorer (the old one, this step does not work with other browsers)
-	In Windows Explorer, select the drive letter to connect again.

## 2.	Shortcut via OneDrive
-	Open https://wageningenur4.sharepoint.com/sites/LoobosTeamsite in any browser
-	Select ‘Add shortcut to OneDrive’
-	In your OneDrive (either via the browser or WindowsExplorer), the sharepoint is now accessible. In my case it was via C:\Users\molen050\OneDrive – WageningenUR\Documents.
-	You can either use that path as the datapath or assign OneDrive a special drive letter:
-	Hit the windows key and type ‘cmd’ to open a dos command window. Type: subst L: "c:\Users\molen050\OneDrive – WageningenUR\Documents\"
-	The Sharepoint will now be accessible via O:\

## 3.	Download the data to your own drive
-	Simple, reliable, but you will not get updates on the data.



# B.	Reading the data in Python
In the Folder DataShare/PythonScripts/ there are two files:
- a python script Loobos_ReadData.py and 
- a Jupyter Notebook Loobos_ReadData.ipynb. 
They contain the same set of commands as an example of how to read the data and make a figure using the python package pandas.

Let me know if you have remarks or questions.
Michiel

In [ ]:
# Settings
Username   = 'molen050'
years      = range(2000,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath   = os.path.join('../../') #since this file is 2 dir down from the data it's ../../
print('datapath is set to %s'%datapath)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import cufflinks as cf
from datetime import datetime, timedelta
#import plotly.express as px
#import cufflinks as cf
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
sys.path.append('../') #append the directory above this one to path to be able to find my python modules

from Loobos_Toolbox import Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile

In [ ]:
#my own functions
from Loobos_Toolbox_Dendro_Grwater import Read_Dendrometers,notmatch

In [ ]:
# Read files
df_EC           = Read_LoobosEddFinal    (years,datapath)
df_Stor         = Read_LooStor           (years,datapath)
df_Comb         = Read_LoodatGapfill     (years,datapath)
df_NEE          = Read_Loobos_halfhourly (years,datapath)
df_meteo        = Read_Loobos_meteo      (years,datapath)
df_soil         = Read_Loobos_soil       (years,datapath)
df_profile      = Read_Loobos_profile    (years,datapath)


## Import the dinoloket data (location B32F0088)

In [ ]:
#new location put 88
dir = '../Dinoloket_2024-01-20/Grondwaterstanden_Put/B32F0088001_1_edit.csv'
df_GWS=pd.read_csv(dir,sep=",",index_col=False)
df_GWS=df_GWS.drop(['Locatie','Filternummer','Bijzonderheid','Opmerking'], axis=1)
df_GWS['Peildatum']=df_GWS['Peildatum']+" 12:00"
df_GWS['datetime']=pd.to_datetime(df_GWS['Peildatum'], dayfirst=True, format='mixed')
df_GWS.index = df_GWS['datetime']
df_GWS=df_GWS.drop(['Peildatum'], axis=1)
#df_GWS

## Import dendrometer data (old)

#Loobos_dendrometers
dir = '..\Loobos_dendrometers\I5.csv'
df_dendro=pd.read_csv(dir,sep=",",index_col=False)
df_dendro['date2018']="2018" #this is a bit of a goofy way to add te year to groeiseizoen but it should work
df_dendro['date2017']="2017"
df_dendro['date2018']=df_dendro['date2018'].str.cat(df_dendro['groeiseizoen'], join='left', sep='-')
df_dendro['date2017']=df_dendro['date2017'].str.cat(df_dendro['groeiseizoen'], join='left', sep='-')

df_dendro['date2018']=pd.to_datetime(df_dendro['date2018'],dayfirst=False, format='%Y-%m-%d %H:%M') #format='%d-%m-%Y'
df_dendro['date2017']=pd.to_datetime(df_dendro['date2017'],dayfirst=False, format='%Y-%m-%d %H:%M') #format='%d-%m-%Y'

df_dendro2018=pd.concat([df_dendro['date2018'],df_dendro['jr2018']],axis=1,sort=False)
df_dendro2017=pd.concat([df_dendro['date2017'],df_dendro['jr2017']],axis=1,sort=False)

df_dendro2018.index = df_dendro2018['date2018']
df_dendro2017.index = df_dendro2017['date2017']

df_dendro2018['jr2018']

## Import dendrometer data (using function)

In [ ]:
## function to import all dendro data and append it to the other data
df_dendro = Read_Dendrometers(datapath)

# My plot: function for plotting

In [ ]:
#plotting combinations of variables:
#WUE water per light flux  Bowen unitless GWS meter dendrometer cm and sap flow speed

In [ ]:
#constructing bowen ratio

#construct temp dataframe
df_bowen = pd.concat([df_Comb['H'],df_Comb['LE'],df_Comb['Hour'],df_Comb['Year']],axis=1,sort=False)

#step 2 select for hours between 9am and 4pm
df_bowen = df_bowen.loc[df_bowen.index.hour <= 16]
df_bowen = df_bowen.loc[df_bowen.index.hour >= 9]

#step 3 exclude data where LE or H are below 50
df_bowen = df_bowen.loc[df_bowen['LE'] >= 50]
df_bowen = df_bowen.loc[df_bowen['H'] >= 50]

#step 4 exclude data where months are not april-september (4-9)
df_bowen = df_bowen.loc[df_bowen.index.month < 10]
df_bowen = df_bowen.loc[df_bowen.index.month > 3]

#step 4 calulcate bowen ratio
df_bowen['Bowen_R']=df_bowen['H']/df_bowen['LE']
#simple moving average. Use the alias '3D' as a string to refer to timedelta = 3 days
#documentation for the method 'rolling': https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
#list of aliases for timedeltas: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
df_bowen['Bowen_R_3DSMA']=df_bowen['Bowen_R'].rolling('3D').mean()

In [ ]:
#water use efficiency
#step 1: create new dataframe with data of interest
df_WUE = pd.concat([df_Comb['GPP_f'],df_Comb['LE'],df_Comb['Hour'],df_Comb['Year']],axis=1,sort=False)

#step 2 select for hours between 9am and 4pm
df_WUE = df_WUE.loc[df_WUE.index.hour <= 16]
df_WUE = df_WUE.loc[df_WUE.index.hour >= 9]

#step 3 exclude data where LE or H are below 50
df_WUE = df_WUE.loc[df_WUE['LE'] >= 100]

#step 4 exclude data where months are not april-september (4-9)
df_WUE = df_WUE.loc[df_WUE.index.month < 10]
df_WUE = df_WUE.loc[df_WUE.index.month > 3]

#step 5 calulcate water use efficiency
df_WUE['WaterUseEff']=df_WUE['GPP_f']/df_WUE['LE']

#remove entries where WUE is less than 0
df_WUE.loc[df_WUE['WaterUseEff']<=0.0] = np.NaN

df_WUE['WaterUseEff_3DSMA']=df_WUE['WaterUseEff'].rolling('3D').mean()
#df_WUE

In [ ]:
#light use efficiency
#step 1: create new dataframe with data of interest
df_LUE = pd.concat([df_meteo['S(i)'],df_Comb['GPP_f'],df_Comb['Hour'],df_Comb['Year']],axis=1,sort=False)

#step 2 select for hours between 9am and 4pm
df_LUE = df_LUE.loc[df_LUE.index.hour <= 16]
df_LUE = df_LUE.loc[df_LUE.index.hour >= 9]

#step 3 exclude data where LE or H are below 50
df_WUE = df_WUE.loc[df_LUE['S(i)'] >= 50]

#step 4 exclude data where months are not april-september (4-9)
df_LUE = df_LUE.loc[df_LUE.index.month < 10]
df_LUE = df_LUE.loc[df_LUE.index.month > 3]

#step 5 calulcate water use efficiency
df_LUE['LightUseEff']=df_LUE['GPP_f']/df_LUE['S(i)']

#remove entries where WUE is less than 0
df_LUE.loc[df_LUE['LightUseEff']<=0.0] = np.NaN

df_LUE['LightUseEff_3DSMA']=df_LUE['LightUseEff'].rolling('3D').mean()
#df_WUE

#use this to test
#df_LUE['LightUseEff_3DSMA'].loc['04-01-2008':'05-01-2008'].plot()

In [ ]:
#VPD
df_VPD = pd.concat([df_Comb['VPD'],df_Comb['Hour'],df_Comb['Year']],axis=1,sort=False)
df_VPD=df_VPD.loc[df_VPD['VPD']>-100]

df_VPD = df_VPD.loc[df_VPD.index.month < 10]
df_VPD = df_VPD.loc[df_VPD.index.month > 3]

df_VPD['VPD_3DSMA']=df_VPD['VPD'].rolling('3D').mean()

In [ ]:
#Precip
df_meteo['P(mast)'].resample('3D').sum()

In [ ]:
#cell used to quickly check start and end of dataframe
#df_Comb.index.min
#df_Comb runs from '1997-01-01 00:00:00' to '2020-12-31 23:30:00'

In [ ]:
#collect all of our new series in a data dataframe
data = pd.DataFrame(np.nan,
                  columns=["empty"],
                  index=pd.date_range(start="19970101 00:00",end="20201231 23:30", freq="30min"))
data=pd.concat([data['empty'],
                df_bowen['Bowen_R'],
                df_bowen['Bowen_R_3DSMA'],
                df_WUE['WaterUseEff'],
                df_WUE['WaterUseEff_3DSMA'],
                df_LUE['LightUseEff'],
                df_LUE['LightUseEff_3DSMA'],
                df_VPD['VPD'],
                df_VPD['VPD_3DSMA'],
                df_bowen['H'],df_bowen['LE'],
                df_WUE['GPP_f'],
                df_GWS['Stand (cm t.o.v. MV)'],
                df_dendro['dendro_growth_I5']],axis=1,sort=False)


#data

In [ ]:
#cell for generating some dummy data
#reminder to self: this is how you index to select data from a dataframe where the index is datetime objects
#dummy_data["LE"]["1998-01-20 10:30"]=0
#dummy_data["1998-01-20 10:00":"1998-01-21 18:30"]["LE"]

In [ ]:
#dictionary specifying the shorthand labels for different variables
#input key is the name of the column in the dataframe. 
#Entries are tuples, first value is shorthand for legend, second for y-axis label
shorthand_labels={
    'Bowen_R':('bowen_r','bowen ratio'),
    'Bowen_R_3DSMA':('bowen_r3day','bowen ratio 3dayAvg'),
    'WaterUseEff':('WUE','WaterUseEfficency'),
    'WaterUseEff_3DSMA':('WUE_3day','Water Use Eff 3dayAvg'),
    'LightUseEff':('LUE','LightUseEfficency'),
    'LightUseEff_3DSMA':('LUE_3day','Light Use Eff 3dayAvg'),
    'H':('sensible_flux','sensible (H) flux'),
    'LE':('latent_flux','latent (LE) flux'),
    'GPP_f':('GPP_flux','GPP flux'),
    'VPD':('VPD','VPD'),
    'VPD_3DSMA':('VPD_3Day','VPD_3Day'),
    'Stand (cm t.o.v. MV)':('GWS','GWS'),
    'dendro_growth_I5':('dendrometer I5','dendrometer I5') }

In [ ]:
def vis_wrapper_drought(time_start,time_end, plot_variables,yaxis_min_maxes,yaxes_auto,data,savefig,figtitle="",resample=""):
    #check if parameters are properly formatted or else interrupt function

    #filter data based on time_start and time_end
    data=data.loc[time_start:time_end]
    
    #filter the data to remove non-summer months and known extreme peaks
    #(I've decided to do this operation outside of this function so that I dont have to re-filter every time, and just pass a data object as a parameter)
    #in the future I might re-migrate it into here and use a plot_update function to switch what the plot is looking at

    #after filtering, check if data is NaN for this timerange, or if there are very few data points.
    #note to self: isnull() function returns a series with True if value is NaN in that element or False if anything else (including 0). Then the .all() does equivalent to boolean AND. So if that returns true, ALL VALUES are NaN.
    for p_var in plot_variables:
        if data[p_var].isnull().all():
            print('warning:your data is all NaNs')


    #resampling the data based on timestep
    if resample!="":
        data=data.resample(resample).mean()

    
    #plotting section of the function
    fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
    fig.subplots_adjust(right=0.75)
    
    #the ax.plot() function in following line works if you feed it a list of strings with column names, so for now leave like this.
    #in future, maybe add support for exception catching so you can make a better error message for the user. Currently its just Keyerror

    #construct twin axes for supporting up to 3 lines with different y axes
    twin1 = ax.twinx()
    twin2 = ax.twinx()
    twin3 = ax.twinx()

    # Offset the right spine of twin2 and twin3.  The ticks and label have already been placed on the right by twinx above.
    twin2.spines.right.set_position(("axes", 1.05))
    twin3.spines.right.set_position(("axes", 1.09))

    #plot values
    p1, = ax.plot(data[plot_variables[0]], "b-", label=shorthand_labels[plot_variables[0]][0])
    p2, = twin1.plot(data[plot_variables[1]], "r-", label=shorthand_labels[plot_variables[1]][0])
    p3, = twin2.plot(data[plot_variables[2]], "g-", label=shorthand_labels[plot_variables[2]][0])
    p4, = twin3.plot(data[plot_variables[3]], "m-", label=shorthand_labels[plot_variables[3]][0])

    #invert y axis because it's GWS
    
    
    if yaxes_auto==False:
        ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
        twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])
        twin2.set_ylim(yaxis_min_maxes[2][0],yaxis_min_maxes[2][1])
        twin3.set_ylim(yaxis_min_maxes[3][0],yaxis_min_maxes[3][1])

    ax.invert_yaxis()
    #set axis labels
    ax.set_xlabel("Time")
    ax.set_ylabel(shorthand_labels[plot_variables[0]][1])
    twin1.set_ylabel(shorthand_labels[plot_variables[1]][1])
    twin2.set_ylabel(shorthand_labels[plot_variables[2]][1])
    twin3.set_ylabel(shorthand_labels[plot_variables[3]][1])

    #color the yaxis labels
    ax.yaxis.label.set_color(p1.get_color())
    twin1.yaxis.label.set_color(p2.get_color())
    twin2.yaxis.label.set_color(p3.get_color())
    twin3.yaxis.label.set_color(p4.get_color())
    #color the rest of the yaxis lines and ticks
    tkw = dict(size=4, width=1.5)
    ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
    twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
    twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
    twin3.tick_params(axis='y', colors=p4.get_color(), **tkw)
    ax.tick_params(axis='x', **tkw)
    
    fig.autofmt_xdate() #allows you to auto-align the x axis labels
    fig.suptitle(figtitle)

    ax.legend(handles=[p1, p2, p3, p4])
    #for future: add support for individual ylimits for each line you added.
    #ax.set_ylim(-0.1,0.3)

    #for future: if savefig is true, then save the figure
    return

In [ ]:
#making a second verion of this function just to plot H and LE
def vis_wrapper_dendro(time_start,time_end, plot_variables,yaxis_min_maxes,yaxes_auto,data,savefig,figtitle="",resample="",P_resample="3D",wid=1):
    #check if parameters are properly formatted or else interrupt function

    #filter data based on time_start and time_end
    data=data.loc[time_start:time_end]
    
    #filter the data to remove non-summer months and known extreme peaks
    #(I've decided to do this operation outside of this function so that I dont have to re-filter every time, and just pass a data object as a parameter)
    #in the future I might re-migrate it into here and use a plot_update function to switch what the plot is looking at

    #after filtering, check if data is NaN for this timerange, or if there are very few data points.
    #note to self: isnull() function returns a series with True if value is NaN in that element or False if anything else (including 0). Then the .all() does equivalent to boolean AND. So if that returns true, ALL VALUES are NaN.
    for p_var in plot_variables:
        if data[p_var].isnull().all():
            print('warning:your data is all NaNs')


    #resampling the data based on timestep
    if resample!="":
        data=data.resample(resample).mean()

    #resampling the precipitation data based on timestep
    if P_resample!="":
        df_Prec = df_meteo['P(mast)'].loc[time_start:time_end].resample(P_resample).sum()
    
    #plotting section of the function
    fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
    fig.subplots_adjust(right=0.75)
    
    #the ax.plot() function in following line works if you feed it a list of strings with column names, so for now leave like this.
    #in future, maybe add support for exception catching so you can make a better error message for the user. Currently its just Keyerror

    #construct twin axes for supporting up to 2 lines with different y axes
    twin1 = ax.twinx()
    twin2 = ax.twinx()
    twin3 = ax.twinx()
    
    # Offset the right spines
    twin2.spines.right.set_position(("axes", 1.05))
    twin3.spines.right.set_position(("axes", 1.10))
    

    #plot values
    p1 = ax.bar(df_Prec.index,df_Prec.values,width=wid, color="b", label="Precip")[0]
    
    p2, = twin1.plot(data[plot_variables[0]], "c-", label="GWL")
    p3, = twin2.plot(data[plot_variables[1]], "g-", label="Dendro")
    p4, = twin3.plot(data[plot_variables[2]], "r-", label="Bowen_R")

    
    if yaxes_auto==False:
        ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
        twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])
        twin2.set_ylim(yaxis_min_maxes[2][0],yaxis_min_maxes[2][1])
        twin3.set_ylim(yaxis_min_maxes[3][0],yaxis_min_maxes[3][1])
        
    twin1.invert_yaxis()
    #set axis labels
    ax.set_xlabel("Time")
    ax.set_ylabel("Precipitation [mm]")
    twin1.set_ylabel("Groundwater level [cm]")
    twin2.set_ylabel("Dendrometer [mm]")
    twin3.set_ylabel("Bowen Ratio[-]")
    ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='2M'))

    #color the yaxis labels
    ax.yaxis.label.set_color('black')
    twin1.yaxis.label.set_color('black')
    twin3.yaxis.label.set_color('black')
    #color the rest of the yaxis lines and ticks
    tkw = dict(size=4, width=1.5)
    ax.tick_params(axis='y', colors='black', **tkw)
    twin1.tick_params(axis='y', colors='black', **tkw)
    ax.tick_params(axis='x', **tkw)
    fig.suptitle(figtitle)
    
    fig.autofmt_xdate() #allows you to auto-align the x axis labels

    ax.legend(handles=[p1, p2, p3 ,p4],labels=['Precip_14day','GWL','Dendro','Bowen_r'])
    #for future: add support for individual ylimits for each line you added.
    #ax.set_ylim(-0.1,0.3)

    #for future: if savefig is true, then save the figure
    return

In [ ]:
#making a second verion of this function just to plot H and LE
def vis_wrapper_dendro3(time_start,time_end, plot_variables,yaxis_min_maxes,yaxes_auto,data,savefig,figtitle="",resample="",P_resample="3D",wid=1):
    #check if parameters are properly formatted or else interrupt function

    #filter data based on time_start and time_end
    data=data.loc[time_start:time_end]
    
    #filter the data to remove non-summer months and known extreme peaks
    #(I've decided to do this operation outside of this function so that I dont have to re-filter every time, and just pass a data object as a parameter)
    #in the future I might re-migrate it into here and use a plot_update function to switch what the plot is looking at

    #after filtering, check if data is NaN for this timerange, or if there are very few data points.
    #note to self: isnull() function returns a series with True if value is NaN in that element or False if anything else (including 0). Then the .all() does equivalent to boolean AND. So if that returns true, ALL VALUES are NaN.
    for p_var in plot_variables:
        if data[p_var].isnull().all():
            print('warning:your data is all NaNs')


    #resampling the data based on timestep
    if resample!="":
        data=data.resample(resample).mean()

    #resampling the precipitation data based on timestep
    if P_resample!="":
        df_Prec = df_meteo['P(mast)'].loc[time_start:time_end].resample(P_resample).sum()
    
    #plotting section of the function
    fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
    fig.subplots_adjust(right=0.75)
    
    #the ax.plot() function in following line works if you feed it a list of strings with column names, so for now leave like this.
    #in future, maybe add support for exception catching so you can make a better error message for the user. Currently its just Keyerror

    #construct twin axes for supporting up to 2 lines with different y axes
    twin1 = ax.twinx()
    twin2 = ax.twinx()
    twin3 = ax.twinx()
    twin4 = ax.twinx()
    
    # Offset the right spines
    twin2.spines.right.set_position(("axes", 1.05))
    twin3.spines.right.set_position(("axes", 1.10))
    twin4.spines.right.set_position(("axes", 1.15))

    #plot values
    p1 = ax.bar(df_Prec.index,df_Prec.values,width=wid, color="b", label="Precip")[0]
    p2, = twin1.plot(data[plot_variables[0]], "c-", label="Groundwater_D")
    p3, = twin2.plot(data[plot_variables[1]], "g-", label="Dendro")
    p4, = twin3.plot(data[plot_variables[2]], "r-", label="Bowen_R")
    p5, = twin4.plot(data[plot_variables[3]], "m-", label="VPD")
    
    if yaxes_auto==False:
        ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
        twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])
        twin2.set_ylim(yaxis_min_maxes[2][0],yaxis_min_maxes[2][1])
        twin3.set_ylim(yaxis_min_maxes[3][0],yaxis_min_maxes[3][1])
        twin4.set_ylim(yaxis_min_maxes[4][0],yaxis_min_maxes[4][1])
        
    twin1.invert_yaxis()
    #set axis labels
    ax.set_xlabel("Time")
    ax.set_ylabel("Precipitation [mm]")
    twin1.set_ylabel("Groundwater depth [cm]")
    twin2.set_ylabel("Dendrometer circumf [mm]")
    twin3.set_ylabel("Bowen Ratio[-]")
    twin4.set_ylabel("VPD [hPa]")
    ax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=[1]))
    #ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='1M'))

    #color the yaxis labels
    ax.yaxis.label.set_color('black')
    twin1.yaxis.label.set_color('black')
    twin2.yaxis.label.set_color('black')
    twin3.yaxis.label.set_color('black')
    twin4.yaxis.label.set_color('black')
    #color the rest of the yaxis lines and ticks
    tkw = dict(size=4, width=1.5)
    ax.tick_params(axis='y', colors='black', **tkw)
    twin1.tick_params(axis='y', colors='black', **tkw)
    twin2.tick_params(axis='y', colors='black', **tkw)
    twin3.tick_params(axis='y', colors='black', **tkw)
    twin4.tick_params(axis='y', colors='black', **tkw)
    ax.tick_params(axis='x', **tkw)
    fig.suptitle(figtitle)
    
    fig.autofmt_xdate() #allows you to auto-align the x axis labels

    ax.legend(handles=[p1, p2, p3 ,p4,p5],labels=['Precip_month','Groundwater_D','Dendro','Bowen_R','VPD'])
    #for future: add support for individual ylimits for each line you added.
    #ax.set_ylim(-0.1,0.3)

    #for future: if savefig is true, then save the figure
    return

In [ ]:
#making a second verion of this function just to plot H and LE
def vis_wrapper_dendro2(time_start,time_end, plot_variables,yaxis_min_maxes,yaxes_auto,data,savefig,figtitle="",resample="",P_resample="3D",wid=1):
    #check if parameters are properly formatted or else interrupt function

    #filter data based on time_start and time_end
    data=data.loc[time_start:time_end]
    
    #filter the data to remove non-summer months and known extreme peaks
    #(I've decided to do this operation outside of this function so that I dont have to re-filter every time, and just pass a data object as a parameter)
    #in the future I might re-migrate it into here and use a plot_update function to switch what the plot is looking at

    #after filtering, check if data is NaN for this timerange, or if there are very few data points.
    #note to self: isnull() function returns a series with True if value is NaN in that element or False if anything else (including 0). Then the .all() does equivalent to boolean AND. So if that returns true, ALL VALUES are NaN.
    for p_var in plot_variables:
        if data[p_var].isnull().all():
            print('warning:your data is all NaNs')


    #resampling the data based on timestep
    if resample!="":
        data=data.resample(resample).mean()

    #resampling the precipitation data based on timestep
    if P_resample!="":
        df_Prec = df_meteo['P(mast)'].loc[time_start:time_end].resample(P_resample).sum()
    
    #plotting section of the function
    fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
    fig.subplots_adjust(right=0.75)
    
    #the ax.plot() function in following line works if you feed it a list of strings with column names, so for now leave like this.
    #in future, maybe add support for exception catching so you can make a better error message for the user. Currently its just Keyerror

    #construct twin axes for supporting up to 2 lines with different y axes
    twin1 = ax.twinx()
    twin2 = ax.twinx()
    twin3 = ax.twinx()
    twin4 = ax.twinx()
    
    # Offset the right spines
    twin2.spines.right.set_position(("axes", 1.05))
    twin3.spines.right.set_position(("axes", 1.10))
    twin4.spines.right.set_position(("axes", 1.15))

    #plot values
    p1 = ax.bar(df_Prec.index,df_Prec.values,width=wid, color="b", label="Precip")[0]
    p2, = twin1.plot(data[plot_variables[0]], "g-", label="Dendro")
    p3, = twin2.plot(data[plot_variables[1]], "r-", label="Bowen_R")
    p4, = twin3.plot(data[plot_variables[2]], "y-", label="LightUseEff")
    p5, = twin4.plot(data[plot_variables[3]], "m-", label="WaterUseEff")
    
    if yaxes_auto==False:
        ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
        twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])
        twin2.set_ylim(yaxis_min_maxes[2][0],yaxis_min_maxes[2][1])
        twin3.set_ylim(yaxis_min_maxes[3][0],yaxis_min_maxes[3][1])
        twin4.set_ylim(yaxis_min_maxes[4][0],yaxis_min_maxes[4][1])
        
    #twin1.invert_yaxis()
    #set axis labels
    ax.set_xlabel("Time")
    ax.set_ylabel("Precipitation [mm]")
    twin1.set_ylabel("Dendrometer [mm]")
    twin2.set_ylabel("Bowen Ratio[-]")
    twin3.set_ylabel("Light Use Eff [μmols-1W-1]")
    twin4.set_ylabel("Water Use Eff [μmols-1W-1]")
    ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='1M'))

    #color the yaxis labels
    ax.yaxis.label.set_color('black')
    twin1.yaxis.label.set_color('black')
    twin2.yaxis.label.set_color('black')
    twin3.yaxis.label.set_color('black')
    twin4.yaxis.label.set_color('black')
    #color the rest of the yaxis lines and ticks
    tkw = dict(size=4, width=1.5)
    ax.tick_params(axis='y', colors='black', **tkw)
    twin1.tick_params(axis='y', colors='black', **tkw)
    twin2.tick_params(axis='y', colors='black', **tkw)
    twin3.tick_params(axis='y', colors='black', **tkw)
    twin4.tick_params(axis='y', colors='black', **tkw)
    ax.tick_params(axis='x', **tkw)
    fig.suptitle(figtitle)
    
    fig.autofmt_xdate() #allows you to auto-align the x axis labels

    ax.legend(handles=[p1, p2, p3 ,p4,p5],labels=['Precip_14day','Dendro','Bowen_R','LUE','WUE'])
    #for future: add support for individual ylimits for each line you added.
    #ax.set_ylim(-0.1,0.3)

    #for future: if savefig is true, then save the figure
    return

In [ ]:
plt.rcParams["figure.figsize"] = (20,3)

In [ ]:
vis_wrapper_dendro(time_start="2017-01-01 00:00",
            time_end="2020-01-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","dendro_growth_I5","Bowen_R_3DSMA"],
            yaxis_min_maxes=[[0,80],[-1,8],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            figtitle='Groundwater level, precipitation, bowen ratio and dendrometer circumference for 2017-2019',
            resample="D",
            P_resample="14D",
            wid=2.5)
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_dendro(time_start="2017-01-01 00:00",
            time_end="2020-01-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","dendro_growth_I5","Bowen_R_3DSMA"],
            yaxis_min_maxes=[[0,80],[-1,8],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            figtitle='Groundwater level, precipitation, bowen ratio and dendrometer circumference for 2017-2019',
            resample="D",
            P_resample="1M",
            wid=4)
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_dendro(time_start="2016-01-01 00:00",
            time_end="2020-01-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","dendro_growth_I5","Bowen_R_3DSMA"],
            yaxis_min_maxes=[[0,80],[-1,8],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            figtitle='Groundwater level, precipitation, bowen ratio and dendrometer circumference for 2017-2019',
            resample="D",
            P_resample="1M",
            wid=4)
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20,6)

In [ ]:
vis_wrapper_dendro3(time_start="2017-03-31 00:00",
            time_end="2019-12-30 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","dendro_growth_I5","Bowen_R_3DSMA","VPD_3DSMA"],
            yaxis_min_maxes=[[0,125],[155,320],[-4,24],[-5,6],[-10,23]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle='Precipitation, Groundwater depth, dendrometer circumference, bowen ratio - for Growth seasons 2017-2019',
            resample="D",
            P_resample="1M",
            wid=4)
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
import matplotlib.dates as mdates

In [ ]:
data['VPD_3DSMA'].plot(ylim=[0,10])

In [ ]:
vis_wrapper_dendro2(time_start="2017-03-31 00:00",
            time_end="2017-09-30 00:00",
            plot_variables=["dendro_growth_I5","Bowen_R_3DSMA","LightUseEff_3DSMA","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[0,90],[-1,6],[0,6],[0,0.10],[0,0.3]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle='Precipitation, bowen ratio, water and light use efficiencies and dendrometer circumference for April-September 2017',
            resample="D",
            P_resample="14D",
            wid=2.5)
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_dendro2(time_start="2018-03-31 00:00",
            time_end="2018-09-30 00:00",
            plot_variables=["dendro_growth_I5","Bowen_R_3DSMA","LightUseEff_3DSMA","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[0,90],[-1,6],[0,6],[0,0.10],[0,0.3]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle='Precipitation, bowen ratio, water and light use efficiencies and dendrometer circumference for April-September 2018',
            resample="D",
            P_resample="14D",
            wid=2.5)
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_dendro2(time_start="2019-03-31 00:00",
            time_end="2019-09-30 00:00",
            plot_variables=["dendro_growth_I5","Bowen_R_3DSMA","LightUseEff_3DSMA","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[280,200],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            figtitle='Precipitation, bowen ratio, water and light use efficiencies and dendrometer circumference for April-September 2019',
            resample="D",
            P_resample="14D",
            wid=2.5)
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_dendro(time_start="2012-01-01 00:00",
            time_end="2017-01-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","dendro_growth_I5"],
            yaxis_min_maxes=[[280,200],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D",
            P_resample="14D",
            wid=2.5)
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
df_Prec = df_meteo['P(mast)'].resample('3D').sum()

df_Prec=df_Prec.loc['2010-01-01':'2010-02-01']

fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True

#p1, = ax.plot(data[plot_variables[0]], "b-", label="GWL")
#p2, = twin1.plot(data[plot_variables[1]], "g-", label="Dendro")
p3 = ax.bar(df_Prec.index,df_Prec.values, label="P")[0]
   
plt.show()

In [ ]:
df_Prec.values

### 2008-2009

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2008-04-01 00:00",
            time_end="2008-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[280,200],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()
#df_GWS['Stand (cm t.o.v. MV)']

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2009-04-01 00:00",
            time_end="2009-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()
#df_GWS['Stand (cm t.o.v. MV)']

### 2013-2014

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2013-04-01 00:00",
            time_end="2013-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()
#df_GWS['Stand (cm t.o.v. MV)']

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2014-04-01 00:00",
            time_end="2014-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()
#df_GWS['Stand (cm t.o.v. MV)']

### 2018-2019

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2018-04-01 00:00",
            time_end="2018-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            #plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()
#df_GWS['Stand (cm t.o.v. MV)']

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2019-04-01 00:00",
            time_end="2019-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()
#df_GWS['Stand (cm t.o.v. MV)']

### inspecting components of bowen ratio

In [ ]:
#testing the function for just H and LE
vis_wrapper_heat(
            time_start="19970101 00:00",
            time_end="20201231 23:30",
            plot_variables=["LE","H"],
            yaxis_min_maxes=[[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="M")
#for resample, "" is no resampling "D" for day.

## Light use Efficiency

## water dependency graphs for 2008-2020

### watertekort -> huidmondjes dicht for 2008-2020

In [ ]:
vis_wrapper_drought(time_start="2008-04-01 00:00",
            time_end="2008-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2008",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2009-04-01 00:00",
            time_end="2009-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2009",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2010-04-01 00:00",
            time_end="2010-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2010",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2011-04-01 00:00",
            time_end="2011-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2011",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2012-04-01 00:00",
            time_end="2012-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2012",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2013-04-01 00:00",
            time_end="2013-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2013",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2014-04-01 00:00",
            time_end="2014-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2014",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2015-04-01 00:00",
            time_end="2015-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2015",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2016-04-01 00:00",
            time_end="2016-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2016",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2017-04-01 00:00",
            time_end="2017-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2017",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2018-04-01 00:00",
            time_end="2018-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2018",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2019-04-01 00:00",
            time_end="2019-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2019",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2020-04-01 00:00",
            time_end="2020-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.5,4.5],[-1.0,10.0],[0,0.2]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="apr-sep 2020",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

## non-water dependency graphs 2008-2020

### geen licht -> geen GPP of droge lucht -> huidmondjes dicht for 2008-2020

In [ ]:
vis_wrapper_drought(time_start="2008-04-01 00:00",
            time_end="2008-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2008",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2009-04-01 00:00",
            time_end="2009-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2009",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2010-04-01 00:00",
            time_end="2010-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2010",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2011-04-01 00:00",
            time_end="2011-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2011",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2012-04-01 00:00",
            time_end="2012-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2012",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2013-04-01 00:00",
            time_end="2013-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2013",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2014-04-01 00:00",
            time_end="2014-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2014",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2015-04-01 00:00",
            time_end="2015-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2015",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2016-04-01 00:00",
            time_end="2016-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2016",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2017-04-01 00:00",
            time_end="2017-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2017",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2018-04-01 00:00",
            time_end="2018-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2018",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2019-04-01 00:00",
            time_end="2019-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2019",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
vis_wrapper_drought(time_start="2020-04-01 00:00",
            time_end="2020-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","LightUseEff_3DSMA","dendro_growth_I5","VPD_3DSMA"],
            yaxis_min_maxes=[[100,300],[0.0,0.10],[-1.0,10.0],[0.0,22.0]],
            yaxes_auto=False,
            data=data,
            savefig=False,
            figtitle="GWS LUE dendro VPD apr-sep 2020",
            resample="D")
#for resample, "" is no resampling "D" for day.
plt.grid()
plt.show()

In [ ]:
df_Comb.columns

In [ ]:
df_meteo.columns

### Present dendro data

In [ ]:
fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
fig.subplots_adjust(right=0.75)
    
#plot values
p1, = ax.plot(, "g-", label="Dendro")
p2, = ax.plot(, "r-", label="Bowen_R")
p3, = ax.plot(, "y-", label="LightUseEff")
p4, = ax.plot(, "m-", label="WaterUseEff")
    

ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
        
#set axis labels
ax.set_xlabel("Time")
ax.set_ylabel("Circumference increase [mm]")
ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='1M'))

#color the yaxis labels
ax.yaxis.label.set_color('black')
#color the rest of the yaxis lines and ticks
tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors='black', **tkw)
ax.tick_params(axis='x', **tkw)
fig.suptitle(figtitle)
    
fig.autofmt_xdate() #allows you to auto-align the x axis labels

ax.legend(handles=[p1, p2, p3 ,p4],labels=['2008','2009','2010','2011'])

plt.grid()
plt.show()

In [ ]:
df_dendro